

`python trainer.py --db sleepedf --gpu 0 --from_fold 0 --to_fold 19`


In [1]:
import warnings

warnings.filterwarnings("ignore")
import argparse
import importlib
import os

In [2]:
from train import train

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [7]:
# Check if CUDA is available
import torch

print("Checking CUDA status...")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"Memory Allocated: {torch.cuda.memory_allocated(i)} bytes")
        print(f"Memory Reserved: {torch.cuda.memory_reserved(i)} bytes")
else:
    print("CUDA is not available. Please check your environment or allocation.")


# Optionally, print out the working directory to verify the file locations
print(f"Current working directory: {os.getcwd()}")

Checking CUDA status...
CUDA available: False
Number of GPUs: 0
CUDA is not available. Please check your environment or allocation.
Current working directory: /cfs/earth/scratch/dugua001/Tinysleepnet-pytorch


In [4]:
args = "Namespace(db='sleepedf', from_fold=0, gpu=0, n_epochs=200, random_seed=42, suffix='', test_batch_size=15, test_seq_len=20, to_fold=19)"
db = "sleepedf"
gpu = 0
from_fold = 0
to_fold = 19
suffix = ""
random_seed = 42



In [5]:
config_file = os.path.join("config", f"{db}.py")
spec = importlib.util.spec_from_file_location("*", config_file)
config = importlib.util.module_from_spec(spec)
spec.loader.exec_module(config)

In [6]:
config_file

'config/sleepedf.py'

In [ ]:
# Output directory
output_dir = f"out_{db}{suffix}"
output_dir

In [11]:
fold_idx = 0

In [ ]:
args=args
config_file=config_file
fold_idx=fold_idx
output_dir=os.path.join(output_dir, 'train')
log_file=os.path.join(output_dir, f'train_{gpu}.log')
restart=True
random_seed=random_seed+fold_idx


In [ ]:
train(
    args=args,
    config_file=config_file,
    fold_idx=fold_idx,
    output_dir=os.path.join(output_dir, "train"),
    log_file=os.path.join(output_dir, f"train_{gpu}.log"),
    restart=True,
    random_seed=random_seed + fold_idx,
)

In [2]:
def run(args, db, gpu, from_fold, to_fold, suffix="", random_seed=42):
    # Set GPU visible

    # Config file
    config_file = os.path.join("config", f"{db}.py")
    spec = importlib.util.spec_from_file_location("*", config_file)
    config = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(config)

    # Output directory
    output_dir = f"out_{db}{suffix}"

    assert from_fold <= to_fold
    assert to_fold < config.params["n_folds"]

    # Training
    for fold_idx in range(from_fold, to_fold + 1):
        train(
            args=args,
            config_file=config_file,
            fold_idx=fold_idx,
            output_dir=os.path.join(output_dir, "train"),
            log_file=os.path.join(output_dir, f"train_{gpu}.log"),
            restart=True,
            random_seed=random_seed + fold_idx,
        )

        # Reset tensorflow graph
        # tf.reset_default_graph()

In [ ]:


config_file = os.path.join("config", f"{db}.py")
config_file

In [ ]:
spec = importlib.util.spec_from_file_location("*", config_file)
spec

In [ ]:
output_dir = f"out_{db}{suffix}"
output_dir = os.path.join(output_dir, "train")
output_dir

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--db", type=str, required=True)
    parser.add_argument("--gpu", type=int, required=True)
    parser.add_argument("--from_fold", type=int, required=True)
    parser.add_argument("--to_fold", type=int, required=True)
    parser.add_argument("--suffix", type=str, default="")
    parser.add_argument("--random_seed", type=int, default=42)
    parser.add_argument("--test_seq_len", type=int, default=20)
    parser.add_argument("--test_batch_size", type=int, default=15)
    parser.add_argument("--n_epochs", type=int, default=200)
    args = parser.parse_args()

    print(args)

    run(
        args=args,
        db=args.db,
        gpu=args.gpu,
        from_fold=args.from_fold,
        to_fold=args.to_fold,
        suffix=args.suffix,
        random_seed=args.random_seed,
    )